In [1]:
!pip install openpyxl

Looking in indexes: https://mirrors.aliyun.com/pypi/simple

[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


# 加载1.8B Chat Int4模型

In [3]:
from modelscope import snapshot_download
from transformers import AutoModelForCausalLM, AutoTokenizer

model_dir = snapshot_download('qwen/Qwen-1_8B-Chat-Int4')

tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    device_map="auto",
    trust_remote_code=True
).eval()

2024-11-21 14:59:51,450 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0
/usr/local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1602: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Try importing flash-attention for faster inference...
Some weights of the model checkpoint at /mnt/workspace/.cache/modelscope/hub/qwen/Qwen-1_8B-Chat-Int4 were not used when initializing QWenLMHeadModel: ['transformer.h.0.attn.c_proj.bias', 'transformer.h.0.mlp.c_proj.bias', 'transformer.h.0.mlp.w1.bias', 'transformer.h.0.mlp.w2.bias', 'transformer.h.1.attn.c_proj.bias', 'transformer.h.1.mlp.c_proj.bias', 'transformer.h.1.mlp.w1.bias', 'transformer.h.1.mlp.w2.bias', 'transformer.h.10.attn.c_proj.bias', 

In [2]:
from modelscope import snapshot_download
from transformers import AutoModelForCausalLM, AutoTokenizer

# Downloading model checkpoint to a local dir model_dir
# model_dir = snapshot_download('qwen/Qwen-7B')
# model_dir = snapshot_download('qwen/Qwen-7B-Chat')
# model_dir = snapshot_download('qwen/Qwen-14B')
model_dir = snapshot_download('qwen/Qwen-1_8B-Chat')

# Loading local checkpoints
# trust_remote_code is still set as True since we still load codes from local dir instead of transformers
tokenizer = AutoTokenizer.from_pretrained(model_dir, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_dir,
    device_map="auto",
    trust_remote_code=True
).eval()

2024-11-28 15:09:43,797 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0
/usr/local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1602: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
from modelscope import AutoModelForCausalLM, AutoTokenizer
from modelscope import GenerationConfig

model.generation_config = GenerationConfig.from_pretrained("qwen/Qwen-1_8B-Chat", trust_remote_code=True) # 可指定不同的生成长度、top_p等相关超参

response, history = model.chat(tokenizer, "你好", history=None)
print(response)
response, history = model.chat(tokenizer, "浙江的省会在哪里？", history=history) 
print(response)
response, history = model.chat(tokenizer, "它有什么好玩的景点", history=history)
print(response)

2024-11-28 15:10:48,407 - modelscope - WARNING - Model revision not specified, use revision: v1.0.0
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


你好！有什么我可以帮助你的吗？
浙江省的省会是杭州。
杭州有许多有趣的地方，比如西湖、灵隐寺、宋城等。此外，杭州还有许多美丽的自然景观和历史遗迹。


# 提示词工程

In [19]:
# 城市数据
with open('/mnt/workspace/city.txt','r',encoding='utf-8') as fp:
    city_list=fp.readlines()
    city_list=[line.strip().split(' ')[1] for line in city_list]

In [18]:
Q='青岛4月6日下雨么?'

prompt_template='''
给定一句话：“%s”，请你按步骤要求工作。

步骤1：识别这句话中的城市和日期共2个信息
步骤2：根据城市和日期信息，生成JSON字符串，格式为{"city":城市,"date":日期}

请问，这个JSON字符串是：
'''

# 生成SFT微调数据

Qwen的SFT数据格式要求:
```
[
  {
    "id": "identity_0",
    "conversations": [
      {
        "from": "user",
        "value": "你好"
      },
      {
        "from": "assistant",
        "value": "我是一个语言模型，我叫通义千问。"
      }
    ]
  }
]
```

In [7]:
import random
import json
import time 

Q_arr=[]
A_arr=[]

Q_list=[
    ('{city}{year}年{month}月{day}日的天气','%Y-%m-%d'),
    ('{city}{year}年{month}月{day}号的天气','%Y-%m-%d'),
    ('{city}{month}月{day}日的天气','%m-%d'),
    ('{city}{month}月{day}号的天气','%m-%d'),

    ('{year}年{month}月{day}日{city}的天气','%Y-%m-%d'),
    ('{year}年{month}月{day}号{city}的天气','%Y-%m-%d'),
    ('{month}月{day}日{city}的天气','%m-%d'),
    ('{month}月{day}号{city}的天气','%m-%d'),

    ('你们{year}年{month}月{day}日去{city}玩吗？','%Y-%m-%d'),
    ('你们{year}年{month}月{day}号去{city}玩么？','%Y-%m-%d'),
    ('你们{month}月{day}日去{city}玩吗？','%m-%d'),
    ('你们{month}月{day}号去{city}玩吗？','%m-%d'),
]

# 生成一批"1月2号"、"1月2日"、"2023年1月2号", "2023年1月2日", "2023-02-02", "03-02"之类的话术, 教会它做日期转换
for i in range(1000):
    Q=Q_list[random.randint(0,len(Q_list)-1)]
    city=city_list[random.randint(0,len(city_list)-1)]
    year=random.randint(1990,2025)
    month=random.randint(1,12)
    day=random.randint(1,28)
    time_str='{}-{}-{}'.format(year,month,day)
    date_field=time.strftime(Q[1],time.strptime(time_str,'%Y-%m-%d'))
    Q=Q[0].format(city=city,year=year,month=month,day=day) # 问题
    A=json.dumps({'city':city,'date':date_field},ensure_ascii=False)  # 回答

    Q_arr.append(prompt_template%(Q,))
    A_arr.append(A)

import pandas as pd 

df=pd.DataFrame({'Prompt':Q_arr,'Completion':A_arr})
df.to_excel('train.xlsx')

In [20]:
import random
import json
import time
import pandas as pd

# 问句模板列表
Q_list = [
    ('{city}{year}年{month}月{day}日的天气', '%Y-%m-%d'),
    ('{city}{year}年{month}月{day}号的天气', '%Y-%m-%d'),
    ('{city}{month}月{day}日的天气', '%m-%d'),
    ('{city}{month}月{day}号的天气', '%m-%d'),
    ('{year}年{month}月{day}日{city}的天气', '%Y-%m-%d'),
    ('{year}年{month}月{day}号{city}的天气', '%Y-%m-%d'),
    ('{month}月{day}日{city}的天气', '%m-%d'),
    ('{month}月{day}号{city}的天气', '%m-%d'),
    ('你们{year}年{month}月{day}日去{city}玩吗？', '%Y-%m-%d'),
    ('你们{year}年{month}月{day}号去{city}玩么？', '%Y-%m-%d'),
    ('你们{month}月{day}日去{city}玩吗？', '%m-%d'),
    ('你们{month}月{day}号去{city}玩吗？', '%m-%d'),
]

# 初始化问题和答案列表
Q_arr = []
A_arr = []
train_data = []

# 生成数据
for i in range(1000):
    Q = Q_list[random.randint(0, len(Q_list) - 1)]
    city = city_list[random.randint(0, len(city_list) - 1)]
    year = random.randint(1990, 2025)
    month = random.randint(1, 12)
    day = random.randint(1, 28)
    
    time_str = '{}-{}-{}'.format(year, month, day)
    date_field = time.strftime(Q[1], time.strptime(time_str, '%Y-%m-%d'))
    
    # 生成问句和答案
    Q_text = Q[0].format(city=city, year=year, month=month, day=day)
    A = json.dumps({'city': city, 'date': date_field}, ensure_ascii=False)
    
    # 填充数据
    Q_arr.append(prompt_template % (Q_text,))
    A_arr.append(A)
    
    # 构建训练数据样本
    example = {
        'id': 'identity_{}'.format(i),
        'conversations': [
            {'from': 'user', 'value': prompt_template % (Q_text,)},
            {'from': 'assistant', 'value': A}
        ]
    }
    train_data.append(example)

# 保存为Excel文件
df = pd.DataFrame({'Prompt': Q_arr, 'Completion': A_arr})
df.to_excel('train.xlsx', index=False)

# 保存为JSON格式
with open('train.txt', 'w', encoding='utf-8') as fp:
    fp.write(json.dumps(train_data, ensure_ascii=False, indent=2))

In [10]:
!git clone https://github.com/QwenLM/Qwen.git

正克隆到 'Qwen'...
remote: Enumerating objects: 1763, done.
remote: Counting objects: 100% (837/837), done.
remote: Compressing objects: 100% (327/327), done.
remote: Total 1763 (delta 625), reused 594 (delta 495), pack-reused 926 (from 1)
接收对象中: 100% (1763/1763), 35.94 MiB | 16.65 MiB/s, 完成.
处理 delta 中: 100% (1039/1039), 完成.


# 微调模型，生成到output_qwen

bash finetune/finetune_qlora_single_gpu.sh  -m /root/.cache/modelscope/hub/qwen/Qwen-1_8B-Chat-Int4 -d /root/Qwen/train.txt

In [7]:
!bash /mnt/workspace/Qwen/finetune/finetune_qlora_single_gpu.sh -m /mnt/workspace/.cache/modelscope/hub/qwen/Qwen-1_8B-Chat -d /mnt/workspace/Qwen/train.txt


python: can't open file '/mnt/workspace/finetune.py': [Errno 2] No such file or directory


In [ ]:
bash /mnt/workspace/Qwen/finetune/finetune_qlora_single_gpu.sh -m /mnt/workspace/.cache/modelscope/hub/qwen/Qwen-1_8B-Chat -d /mnt/workspace/Qwen/train.txt



# 加载SFT后的模型

In [21]:
from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
    '/mnt/workspace/Qwen/output_qwen', # path to the output directory
    device_map="auto",
    trust_remote_code=True
).eval()

The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [22]:
model.generation_config.top_p=0 # 只选择概率最高的token

Q_list=['2020年4月16号三亚下雨么？','青岛3-15号天气预报','5月6号下雪么，城市是威海','青岛2023年12月30号有雾霾么?','我打算6月1号去北京旅游，请问天气怎么样？','你们打算1月3号坐哪一趟航班去上海？','小明和小红是8月8号在上海结婚么?',
        '一起去东北看冰雕么，大概是1月15号左右，我们3个人一起']
for Q in Q_list:
    prompt=prompt_template%(Q,)
    A,hist=model.chat(tokenizer,prompt,history=None)
    print('Q:%s\nA:%s\n'%(Q,A))

Q:2020年4月16号三亚下雨么？
A:{"city": "三亚", "date": "2020-04-16"}

Q:青岛3-15号天气预报
A:{"city": "青岛", "date": "03-15"}

Q:5月6号下雪么，城市是威海
A:{"city": "威海", "date": "05-06"}

Q:青岛2023年12月30号有雾霾么?
A:{"city": "青岛", "date": "2023-12-30"}

Q:我打算6月1号去北京旅游，请问天气怎么样？
A:{"city": "北京", "date": "06-01"}

Q:你们打算1月3号坐哪一趟航班去上海？
A:{"city": "上海", "date": "01-03"}

Q:小明和小红是8月8号在上海结婚么?
A:{"city": "上海", "date": "08-08"}

Q:一起去东北看冰雕么，大概是1月15号左右，我们3个人一起
A:{"city": "东北", "date": "01-15"}



In [23]:
prompt='青岛海边钓鱼需要特别注意什么？'
resp,hist=model.chat(tokenizer,prompt,history=None)
print(resp)

1. 钓鱼时要选择合适的地点，最好是在有遮挡物的地方，避免阳光直射。
2. 钓鱼时要注意天气变化，及时调整钓鱼姿势和时间。
3. 钓鱼时要保持耐心，不要急于求成。


In [24]:
prompt='东京会什么？'
resp,hist=model.chat(tokenizer,prompt,history=None)
print(resp)

东京是日本的首都，也是该国的政治、经济和文化中心。它位于本州岛中部，东临太平洋，西隔海峡与九州岛相望；气候属于温带海洋性季风气候，四季分明。
